# Pytorch Basics

## Imports

In [309]:
#!pip3 install torch torchvision torchaudio
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
#import torch.nn.functional as F

## 1. Simple NeuralNetwork with Pytorch

### 1.1. Basic Example with MSE Loss function (without Gradient Descent)

In [304]:
def forward(x):
    return x*w

def loss(x,y_true):
    y_pred = forward(x)
    return np.sum((y_pred - y_true)**2)/N

In [305]:
x = np.array([1,2,3,4],dtype=np.float32)
y_true = 2*x
x, y_true

(array([1., 2., 3., 4.], dtype=float32),
 array([2., 4., 6., 8.], dtype=float32))

In [ ]:
N = len(x)
w = 0.5 # random weight

In [ ]:
y_random = forward(x)
y_random

In [ ]:
plt.plot(x,y_true, color="blue", label="True function")
plt.plot(x,y_random, color="green", label="Random-valued function")
plt.legend()

In [ ]:
list_loss = np.array([])
list_w = np.array([])

for w in np.arange(0.0,5.0,1.0):
    print("Loss calculation for w =",w)
    y_pred = forward(x)
    l = loss(x,y_true)
    
    print(f"optimal: {y_true}, x*w= {y_pred}, loss={l}")
    
    if l == 0:
        plt.scatter(w,l, color="orange", label="Loss = 0")
    else:
        plt.scatter(w,l, color="blue", linewidths=0.1)
    
    list_loss = np.append(list_loss,[l])
    list_w = np.append(list_w,[w])
    print(list_loss, list_w)

plt.plot(list_w,list_loss)
plt.xlabel("w")
plt.ylabel("MSE Loss")
plt.legend()

In [ ]:
print(list_loss, list_w)

In [ ]:
np.argmin(list_loss), list_w[np.argmin(list_loss)]

In [ ]:
w # the value for weight is from the for loop as the last one choosen (begin: 0.0 until 4.0)
# we need to assign the right one from one row above with list_w[np.argmin(list_loss)] then for the pred function

### 1.2. Basic Example with MSE Loss function and active Training (with Gradient descent)

In [ ]:
def gradient(x,y_true):
    return 2*x*(x*w-y_true)/N

In [ ]:
x = np.array([1,2,3],dtype=np.float32)
y_true = 2*x
x, y_true

In [ ]:
w = 1.0
lr = 0.01

In [ ]:
print(f"w0 = {w}\nx = {x}\ny={y_true}")

In [ ]:
y_pred = forward(x)
y_pred

In [ ]:
gradient(x,y_true)

In [ ]:
forward(4)

In [ ]:
for epoch in range(100):
    
    for x_val, y_val in zip(x,y_true):
        #print("actual values from x and y_true:",x_val, y_val)
        grad = gradient(x_val, y_val)
        w = w - lr * grad
        l = loss(x_val,y_val)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}:\ngrad (for {x_val, y_val} - last in this x_val and y_val): {grad}\nw: {w}, loss: {l:,.3f}\n")

In [ ]:
forward(4)

#### 1.2.1. Example 1 (with 2 weights)

In [ ]:
def function_exercise(x):
    return 2*x

In [ ]:
x = np.array([1,2,3],dtype=np.float32)
y_true = function_exercise(x)

w1 = 0.5
w2 = 0.5
b = 0.0

#y_pred = x**2*w2+x*w1+b
#y_pred, y_true

In [ ]:
def forward_exercise(x):
    return x**2*w2+x*w1+b

In [ ]:
def gradient_w1(x,y_true):
    return 2*x*(x*w1-y_true+x**2*w2+b)

def gradient_w2(x,y_true):
    return  2*x**2*(x**2*w2-y_true+x*w1+b)

In [ ]:
gradient_w1(x,y_true)

In [ ]:
gradient_w2(x,y_true)

In [ ]:
forward_exercise(x) # function which we will train st. it is equal to y_true (2*x)

In [ ]:
y_true

In [ ]:
for epoch in range(100):
    
    for x_val, y_val in zip(x,y_true):
        
        grad_w1 = gradient_w1(x_val, y_val)
        grad_w2 = gradient_w2(x_val, y_val)
        w1 = w1 - lr * grad_w1
        w2 = w2 - lr * grad_w2
        
        l = loss(x_val,y_val)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}:\ngrad (for {x_val, y_val} - last in this x_val and y_val): {grad}\nw: {w}, loss: {l:,.3f}\n")

In [ ]:
forward_exercise(x) # with more epochs (like 1000) we get a perfect prediction which is equal to y_true (2*x)

In [ ]:
y_true

### 1.3. Backpropagation and Autograd

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0])
y_true = 2*x
x,y_true

In [ ]:
lr = 0.01

In [ ]:
w = torch.tensor([1.0], requires_grad=True)
w

In [ ]:
def forward(x):
    return x*w

def loss(x,y_true):
    y_pred = forward(x)
    return (y_pred - y_true)**2

In [ ]:
forward(x)

In [ ]:
w.data, w # w.grad.data -> calling und using only after .backward() 

In [ ]:
list_loss = np.array([])
list_w = np.array([])

for epoch in range(50):
    
    for x_val, y_val in zip(x,y_true):
        l = loss(x_val,y_val)
        l.backward() # activates w.grad.data and replace grad = gradient(x_val, y_val)
        
        w.data = w.data - lr * w.grad.data
        w.grad.data.zero_() # set the gradients to zero after updating the weights
    
    if epoch % 10 == 0:
        print(f"update:\n weight ({w.data}), weight gradient ({w.grad.data}), loss ({l})")
        
    list_loss = np.append(list_loss,[l.detach().numpy()])
    list_w = np.append(list_w,[w.data.detach().numpy()])

In [ ]:
list_loss, list_w

In [ ]:
plt.plot(list_w,list_loss)
plt.xlabel("w")
plt.ylabel("MSE Loss")

In [ ]:
w, w.data

In [ ]:
forward(x)

In [ ]:
x

In [ ]:
plt.scatter(x.detach().numpy(), forward(x).detach().numpy())

#### 1.3.1. Exercise

In [ ]:
def function_exercise(x):
    return 2*x

def forward_exercise(x):
    return x**2*w2+x*w1+b

def loss(x,y_true):
    y_pred = forward_exercise(x)
    return (y_pred - y_true)**2

x = torch.tensor([1.,2.,3.]) 
y_true = function_exercise(x)

lr = 0.01

w1 = torch.tensor(0.5, requires_grad=True)
w2 = torch.tensor(0.5, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [ ]:
w1.data

In [ ]:
forward_exercise(x)

In [ ]:
y_true

In [ ]:
print(f"Before training:\nPredicted Function with random weights: {forward_exercise(x)}, True Function: {y_true}\n\n") 

for epoch in range(100):
    
    for x_val, y_val in zip(x,y_true):
        l = loss(x_val, y_val)
        l.backward()
        
        w1.data = w1.data - lr * w1.grad.data
        w2.data = w2.data - lr * w2.grad.data
        
        w1.grad.data.zero_()
        w2.grad.data.zero_()
        
    if epoch % 10 == 0:
        print(f"Epoch {epoch}:\nWeights: {w1.data, w2.data}, Output: {forward_exercise(x).data} (True function: {y_true}), Loss: {l:,.3f}\n")

### 1.4. Linear Regression

In [233]:
x_data = torch.tensor([[1.],[2.],[3.0]])
y_data = 2*x_data #torch.tensor([[2.],[4.],[6.0]])
x_data, y_data, x_data.shape

(tensor([[1.],
         [2.],
         [3.]]),
 tensor([[2.],
         [4.],
         [6.]]),
 torch.Size([3, 1]))

In [234]:
# Basic Linear regression model
class LinearRegression(torch.nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        
        self.linear = torch.nn.Linear(1,1)
    
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [235]:
model = LinearRegression()

In [236]:
for param in model.parameters():
    print(param.data)

tensor([[-0.7558]])
tensor([0.6374])


In [237]:
print(model.linear.weight.shape, model.linear.weight.data)

torch.Size([1, 1]) tensor([[-0.7558]])


In [238]:
print(model.linear.bias.shape, model.linear.bias.data)

torch.Size([1]) tensor([0.6374])


In [239]:
for name in model.parameters():
    print(name)

Parameter containing:
tensor([[-0.7558]], requires_grad=True)
Parameter containing:
tensor([0.6374], requires_grad=True)


In [240]:
# Output before training
test_prediction = torch.tensor([[4.]])
print(f"Prediction: {model.forward(test_prediction.data).item():,.4f}")

Prediction: -2.3857


In [241]:
# wx+b
model.linear.weight.data*test_prediction+model.linear.bias.data

tensor([[-2.3857]])

In [242]:
criterion = torch.nn.MSELoss() # good

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
optimizer = torch.optim.Adamax(model.parameters(), lr=0.01)
optimizer = torch.optim.ASGD(model.parameters(), lr=0.01)
# optimizer = torch.optim.LBFGS(model.parameters()) # LBFGS.step() missing 1 required positional argument: 'closure'
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)
optimizer = torch.optim.Rprop(model.parameters(), lr=0.01) # good for this example
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # good for this example

In [243]:
# Training
for epoch in range(500):
    y_pred = model(x_data) # forward pass
    loss = criterion(y_pred, y_data) 
    
    if epoch % 100== 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")
        print(f"Prediction: {model.forward(test_prediction.data).item():,.5f}")
    
    optimizer.zero_grad() # set the gradients to zero
    loss.backward() # backward pass
    optimizer.step() # updates weights

Epoch 0, Loss: 28.820131301879883
Prediction: -2.38569
Epoch 100, Loss: 0.21291345357894897
Prediction: 7.07452
Epoch 200, Loss: 0.13156750798225403
Prediction: 7.27255
Epoch 300, Loss: 0.08130071312189102
Prediction: 7.42816
Epoch 400, Loss: 0.05023894086480141
Prediction: 7.55048


In [244]:
# Output after training
test_prediction = torch.tensor([[4.]])
print(f"Prediction: {model.forward(test_prediction.data).item():,.4f}")

Prediction: 7.6466


In [245]:
# wx+b
model.linear.weight.data*test_prediction+model.linear.bias.data

tensor([[7.6466]])

### 1.5. Logistic Regression

In [258]:
x_data = torch.tensor([[1.],[2.],[3.],[4.],[5.], [6.]])
y_data = torch.tensor([[0.],[0.],[0.],[1.],[1.], [1.]])
x_data.shape

torch.Size([6, 1])

In [362]:
# Basic logistic regression model
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        
        self.linear = torch.nn.Linear(1,1)
        self.s = nn.Sigmoid()
        self.r = nn.ReLU()
        self.t = nn.Tanh()
    
    def forward(self, x):
        y_pred = self.s(self.linear(x))
        return y_pred

In [363]:
model = LogisticRegression()
model

LogisticRegression(
  (linear): Linear(in_features=1, out_features=1, bias=True)
  (s): Sigmoid()
  (r): ReLU()
  (t): Tanh()
)

In [364]:
test_prediction = torch.tensor([[4.]])
model(test_prediction)

tensor([[0.6271]], grad_fn=<SigmoidBackward0>)

In [365]:
criterion = torch.nn.BCELoss()
#criterion = torch.nn.BCEWithLogitsLoss() # for relu, tanh
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [366]:
# Training
for epoch in range(10_000):
    y_pred = model(x_data) # forward pass
    loss = criterion(y_pred, y_data) 
    
    if epoch % 1000== 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")
        print(f"Prediction: {model.forward(test_prediction.data).item():,.5f}")
    
    optimizer.zero_grad() # set the gradients to zero
    loss.backward() # backward pass
    optimizer.step() # updates weights

Epoch 0, Loss: 0.6910417675971985
Prediction: 0.62706
Epoch 1000, Loss: 0.4469590485095978
Prediction: 0.69123
Epoch 2000, Loss: 0.3385319709777832
Prediction: 0.69831
Epoch 3000, Loss: 0.2794446349143982
Prediction: 0.70696
Epoch 4000, Loss: 0.2423749417066574
Prediction: 0.71532
Epoch 5000, Loss: 0.21673928201198578
Prediction: 0.72306
Epoch 6000, Loss: 0.19777274131774902
Prediction: 0.73018
Epoch 7000, Loss: 0.1830432415008545
Prediction: 0.73675
Epoch 8000, Loss: 0.1711837500333786
Prediction: 0.74283
Epoch 9000, Loss: 0.1613667756319046
Prediction: 0.74850


In [367]:
test_prediction = torch.tensor([[4.]])
model(test_prediction)

tensor([[0.7538]], grad_fn=<SigmoidBackward0>)